In [ ]:
#| default_exp tc.simple

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# First TC model

Sample TC model with a trade inpact model that is linear with the size of the number of shares/contracts. Denominator is the median traling volume and the numberator is the trade size. This model has not been calibrated and should not be assumed correct.

In [ ]:
#| export
import pandas as pd
import numpy as np

In [ ]:
#| export
class simple():
    def __init__(self,data,param):
        self.ohlcv = data # this assumes a very specific format... note, more sofisticated code should take this into account
        self.slope = param['tc_slope']
        self.bias  = param['tc_bias']
    def TC(self,trade):
        V = self.ohlcv[:,5]
        p = self.ohlcv[:,4]
        # The data is daily. I assume a 24 session.
        v = V/(24*60)
        vm = np.median(v[-40:])
        dp = np.diff(p)
        ss = np.std(dp[-40:])
        return self.slope*(ss*np.abs(trade)/vm)+self.bias

## Example

In [ ]:
#| eval: false

# Get data
from backtest_sample.data_interface import get_data
import_module = get_data()
df = import_module('BTC-USD').sim_data()

[*********************100%***********************]  1 of 1 completed

In [ ]:
#| eval: false
# TC in the begging
prm = {}
prm['tc_slope'] = 1e3
prm['tc_bias'] = 100
tc = simple(df.values[:100,:],prm)
print(tc.TC(1000))

813.5629449701777
813.5629449701777


In [ ]:
#| eval: false
# TC in the end
prm = {}
prm['tc_slope'] = 1e3
prm['tc_bias'] = 100
tc = simple(df.values[-100:,:],prm)
print(tc.TC(300))

125.52646522553687
125.52646522553687
